## Objective:

To find the context of the data using NLP. Data consist of Twitts from various people from Twitter platform.

In [1]:
import spacy                             # NLP Library
import pandas as pd
import numpy as np
import tensorflow as tf

# spacy.cli.download('en_core_web_lg')   # Download the English language module for text processing
# en_core_web_lg: en --> english | web --> trained on web data | lg --> large

nlp = spacy.load('en_core_web_lg')

In [2]:
# ! unzip Sentiment.csv.zip
df = pd.read_csv('Sentiment.csv')
df

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,...,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,1,No candidate mentioned,1.0000,yes,1.0000,Neutral,0.6578,None of the above,1.0000,NaN,...,NaN,5,NaN,NaN,RT @NancyLeeGrahn: How did everyone feel about...,NaN,2015-08-07 09:54:46 -0700,629697200650592256,NaN,Quito
1,2,Scott Walker,1.0000,yes,1.0000,Positive,0.6333,None of the above,1.0000,NaN,...,NaN,26,NaN,NaN,RT @ScottWalker: Didn't catch the full #GOPdeb...,NaN,2015-08-07 09:54:46 -0700,629697199560069120,NaN,NaN
2,3,No candidate mentioned,1.0000,yes,1.0000,Neutral,0.6629,None of the above,0.6629,NaN,...,NaN,27,NaN,NaN,RT @TJMShow: No mention of Tamir Rice and the ...,NaN,2015-08-07 09:54:46 -0700,629697199312482304,NaN,NaN
3,4,No candidate mentioned,1.0000,yes,1.0000,Positive,1.0000,None of the above,0.7039,NaN,...,NaN,138,NaN,NaN,RT @RobGeorge: That Carly Fiorina is trending ...,NaN,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada)
4,5,Donald Trump,1.0000,yes,1.0000,Positive,0.7045,None of the above,1.0000,NaN,...,NaN,156,NaN,NaN,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,NaN,2015-08-07 09:54:45 -0700,629697196967903232,NaN,Arizona
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13866,13867,No candidate mentioned,1.0000,yes,1.0000,Negative,0.7991,Abortion,0.6014,No candidate mentioned,...,yes,7,Negative,Abortion\nWomen's Issues (not abortion though),RT @cappy_yarbrough: Love to see men who will ...,NaN,2015-08-07 09:29:43 -0700,629690895479250944,Como,NaN
13867,13868,Mike Huckabee,0.9611,yes,1.0000,Positive,0.7302,None of the above,0.9229,Mike Huckabee,...,yes,1,NaN,NaN,RT @georgehenryw: Who thought Huckabee exceede...,NaN,2015-08-07 09:25:02 -0700,629689719056568320,USA,NaN
13868,13869,Ted Cruz,1.0000,yes,1.0000,Positive,0.8051,None of the above,0.9647,Ted Cruz,...,yes,67,Positive\nNeutral,NaN,"RT @Lrihendry: #TedCruz As President, I will a...",NaN,2015-08-07 07:19:18 -0700,629658075784282112,NaN,NaN
13869,13870,Donald Trump,1.0000,yes,1.0000,Negative,1.0000,Women's Issues (not abortion though),0.9202,Donald Trump,...,yes,149,NaN,Women's Issues (not abortion though),RT @JRehling: #GOPDebate Donald Trump says tha...,NaN,2015-08-07 09:54:04 -0700,629697023663546368,NaN,NaN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13871 entries, 0 to 13870
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         13871 non-null  int64  
 1   candidate                  13775 non-null  object 
 2   candidate_confidence       13871 non-null  float64
 3   relevant_yn                13871 non-null  object 
 4   relevant_yn_confidence     13871 non-null  float64
 5   sentiment                  13871 non-null  object 
 6   sentiment_confidence       13871 non-null  float64
 7   subject_matter             13545 non-null  object 
 8   subject_matter_confidence  13871 non-null  float64
 9   candidate_gold             28 non-null     object 
 10  name                       13871 non-null  object 
 11  relevant_yn_gold           32 non-null     object 
 12  retweet_count              13871 non-null  int64  
 13  sentiment_gold             15 non-null     obj

In [5]:
# Extract important features for model

df2 = df.loc[:,["candidate",'text','sentiment']]
df2

,candidate,text,sentiment
0,No candidate mentioned,RT @NancyLeeGrahn: How did everyone feel about...,Neutral
1,Scott Walker,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
2,No candidate mentioned,RT @TJMShow: No mention of Tamir Rice and the ...,Neutral
3,No candidate mentioned,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,Donald Trump,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive
...,...,...,...
13866,No candidate mentioned,RT @cappy_yarbrough: Love to see men who will ...,Negative
13867,Mike Huckabee,RT @georgehenryw: Who thought Huckabee exceede...,Positive
13868,Ted Cruz,"RT @Lrihendry: #TedCruz As President, I will a...",Positive
13869,Donald Trump,RT @JRehling: #GOPDebate Donald Trump says tha...,Negative


In [6]:
# check for class imbalance

df2.sentiment.value_counts()

Negative    8493
Neutral     3142
Positive    2236
Name: sentiment, dtype: int64

In [7]:
df2.candidate.value_counts()

No candidate mentioned    7491
Donald Trump              2813
Jeb Bush                   705
Ted Cruz                   637
Ben Carson                 404
Mike Huckabee              393
Chris Christie             293
Marco Rubio                275
Rand Paul                  263
Scott Walker               259
John Kasich                242
Name: candidate, dtype: int64

In [11]:
# df2.groupby(['candidate','sentiment']).count()
# OR
pd.crosstab(df2.candidate, df2.sentiment)

sentiment,Negative,Neutral,Positive
candidate,,,
Ben Carson,186,54,164
Chris Christie,218,42,33
Donald Trump,1758,446,609
Jeb Bush,589,72,44
John Kasich,82,47,113
Marco Rubio,105,51,119
Mike Huckabee,237,83,73
No candidate mentioned,4724,2087,680
Rand Paul,148,60,55


## Text Preprocessing

In [12]:
df2.text

0        RT @NancyLeeGrahn: How did everyone feel about...
1        RT @ScottWalker: Didn't catch the full #GOPdeb...
2        RT @TJMShow: No mention of Tamir Rice and the ...
3        RT @RobGeorge: That Carly Fiorina is trending ...
4        RT @DanScavino: #GOPDebate w/ @realDonaldTrump...
                               ...                        
13866    RT @cappy_yarbrough: Love to see men who will ...
13867    RT @georgehenryw: Who thought Huckabee exceede...
13868    RT @Lrihendry: #TedCruz As President, I will a...
13869    RT @JRehling: #GOPDebate Donald Trump says tha...
13870    RT @Lrihendry: #TedCruz headed into the Presid...
Name: text, Length: 13871, dtype: object

In [13]:
# Using Regex method to remove unwanted string data.

df2.text = df2.text.str.replace('^RT\s@\w+:\s', '', regex = True)
df2.text

0        How did everyone feel about the Climate Change...
1        Didn't catch the full #GOPdebate last night. H...
2        No mention of Tamir Rice and the #GOPDebate wa...
3        That Carly Fiorina is trending -- hours after ...
4        #GOPDebate w/ @realDonaldTrump delivered the h...
                               ...                        
13866    Love to see men who will never be faced with a...
13867    Who thought Huckabee exceeded their expectatio...
13868    #TedCruz As President, I will always tell the ...
13869    #GOPDebate Donald Trump says that he doesn't h...
13870    #TedCruz headed into the Presidential Debates....
Name: text, Length: 13871, dtype: object

In [15]:
# Removing more unwanted stuff using regex method

df2.text = df2.text.str.lower()    # converting to lower case for simplicity
df2.text[0]

'how did everyone feel about the climate change question last night? exactly. #gopdebate'

In [19]:
nlp = spacy.load('en_core_web_lg')

def remove_unwanted(text):
  newWord = []
  doc = nlp(text)                   # will covert the input texts to NLP Doc for futher processing in NLP

  for word in doc:
    if word.is_stop == False:       # will allow all the words which are not a stopword
      if word.is_punct == False:    # will not allow punctuations, if any
        newWord.append(word.text)   # this will output a list of words
  newWord_str = ' '.join(newWord)   # this will convert coming list of words to a string
  return newWord_str

In [17]:
df2.text[0]

'how did everyone feel about the climate change question last night? exactly. #gopdebate'

In [20]:
remove_unwanted("how did everyone feel about the climate change question last night? exactly. #gopdebate")

'feel climate change question night exactly gopdebate'

In [21]:
# use .apply method to apply the created Function to df2.text

df2["textv2"] = df2.text.apply(remove_unwanted)  # created a new column with made changes
df2

,candidate,text,sentiment,textv2
0,No candidate mentioned,how did everyone feel about the climate change...,Neutral,feel climate change question night exactly gop...
1,Scott Walker,didn't catch the full #gopdebate last night. h...,Positive,catch gopdebate night scott best lines 90 seco...
2,No candidate mentioned,no mention of tamir rice and the #gopdebate wa...,Neutral,mention tamir rice gopdebate held cleveland wow
3,No candidate mentioned,that carly fiorina is trending -- hours after ...,Positive,carly fiorina trending hours debate men comple...
4,Donald Trump,#gopdebate w/ @realdonaldtrump delivered the h...,Positive,gopdebate w/ @realdonaldtrump delivered highes...
...,...,...,...,...
13866,No candidate mentioned,love to see men who will never be faced with a...,Negative,love men faced pregnancy talk body ❤ ️ ❤ ️ ❤ ️...
13867,Mike Huckabee,who thought huckabee exceeded their expectatio...,Positive,thought huckabee exceeded expectations \n\n g...
13868,Ted Cruz,"#tedcruz as president, i will always tell the ...",Positive,tedcruz president tell truth said gopdebates
13869,Donald Trump,#gopdebate donald trump says that he doesn't h...,Negative,gopdebate donald trump says time political cor...


In [22]:
# using "re" library for more filteration

import re

df2.textv2 = df2.textv2.apply(lambda x: re.sub('[^a-zA-Z0-9\s]', '', x))   # here re.sub means Substitute from "re" library, which is equivalent to Replace which we used earlier

In [23]:
df2.sample(5)

,candidate,text,sentiment,textv2
7690,No candidate mentioned,my summary of #gopdebate candidates from last ...,Negative,summary gopdebate candidates night mouth breat...
8569,No candidate mentioned,"i didn't watch the #gopdebates tonight, so i w...",Neutral,watch gopdebates tonight going ask won figured
4435,Donald Trump,raise your hand if you've ever felt personally...,Negative,raise hand felt personally victimized donald t...
7959,No candidate mentioned,love liza's cartoons! #gopdebates https://t.c...,Positive,love liza cartoons gopdebates httpstcovkhqn0...
5947,No candidate mentioned,pretty sure @mtaibbi is directly responsible f...,Negative,pretty sure mtaibbi directly responsible massi...


## Implementing LSTM Model

In [24]:
# Getting the input to the model ready

from tensorflow.keras.preprocessing.text import Tokenizer           # this will assign a number to each word in the text --> indexing
from tensorflow.keras.preprocessing.sequence import pad_sequences   # this will add padding to the shorter texts so that the number of words in text in df2.textv2 is equal

# Part 1: Indexing using Tokenizer

max_feature = 2000
token = Tokenizer(max_feature, split= " ")

token.fit_on_texts(df2.textv2)

In [25]:
token.index_word    # here is the indexing done for each word

{1: 'gopdebate',
 2: 'gopdebates',
 3: 'trump',
 4: 'fox',
 5: 'realdonaldtrump',
 6: 'debate',
 7: 'amp',
 8: 'news',
 9: 'night',
 10: 'candidates',
 11: 'gop',
 12: 'like',
 13: 'megynkelly',
 14: 'people',
 15: 'foxnews',
 16: 'jeb',
 17: 'bush',
 18: 'think',
 19: 'cruz',
 20: 'president',
 21: 'republican',
 22: 'god',
 23: 'donald',
 24: 'need',
 25: 'chris',
 26: 'ask',
 27: 'rubio',
 28: 'want',
 29: 'questions',
 30: 'wallace',
 31: 'carson',
 32: 'question',
 33: 'women',
 34: 'said',
 35: 'time',
 36: 'know',
 37: 'watching',
 38: 'tonight',
 39: 'candidate',
 40: 'huckabee',
 41: 'right',
 42: 'job',
 43: 'tedcruz',
 44: 'fair',
 45: 'megyn',
 46: 'america',
 47: 'trying',
 48: 'got',
 49: 'face',
 50: 'hear',
 51: 'tcot',
 52: 'american',
 53: 'balanced',
 54: 'stage',
 55: 'good',
 56: 'paul',
 57: 'party',
 58: 'watch',
 59: 'hillary',
 60: 'great',
 61: 'walker',
 62: 'kasich',
 63: 'talk',
 64: 'g',
 65: 'money',
 66: 'won',
 67: 'carlyfiorina',
 68: 'field',
 69: 'me

In [26]:
# Part 2: Padding

df2.textv2[0]

'feel climate change question night exactly gopdebate'

In [27]:
df2.textv2[1]

'catch gopdebate night scott best lines 90 seconds walker16 httptcozsff'

In [28]:
x = token.texts_to_sequences(df2.textv2)
x_pad = pad_sequences(x)

In [31]:
x[0]

[288, 256, 214, 32, 9, 801, 1]

In [32]:
x[1]

[1612, 1, 9, 178, 76, 1142, 1281, 1026, 619]

In [36]:
len(x[0]), len(x[1])

(7, 9)

In [29]:
x_pad[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0, 288, 256, 214,  32,   9, 801,   1],
      dtype=int32)

In [30]:
x_pad[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0, 1612,    1,    9,  178,   76, 1142,
       1281, 1026,  619], dtype=int32)

In [37]:
len(x_pad[0]), len(x_pad[1])

(25, 25)

In [38]:
# Getting the Training output to the model ready

df2.sentiment

0         Neutral
1        Positive
2         Neutral
3        Positive
4        Positive
           ...   
13866    Negative
13867    Positive
13868    Positive
13869    Negative
13870    Positive
Name: sentiment, Length: 13871, dtype: object

In [39]:
# this can be done either by using Label Encoding / One hot encoding
# OR
# Pandas encoding --> get_dummies

y = pd.get_dummies(df2.sentiment)
y

,Negative,Neutral,Positive
0,0,1,0
1,0,0,1
2,0,1,0
3,0,0,1
4,0,0,1
...,...,...,...
13866,1,0,0
13867,0,0,1
13868,0,0,1
13869,1,0,0


In [41]:
# Creating model

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(max_feature, 128, input_length = x_pad.shape[1]))
model.add(tf.keras.layers.LSTM(128, dropout=0.3))
model.add(tf.keras.layers.Dense(3, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 25, 128)           256000    
                                                                 
 lstm (LSTM)                 (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 3)                 387       
                                                                 
Total params: 387971 (1.48 MB)
Trainable params: 387971 (1.48 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [42]:
model.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer='adam',
              metrics=['acc'])

In [43]:
model.fit(x_pad, y, epochs=5, validation_split=0.1)

Epoch 1/5
391/391 [==============================] - 16s 35ms/step - loss: 0.8176 - acc: 0.6470 - val_loss: 0.7031 - val_acc: 0.6816
Epoch 2/5
391/391 [==============================] - 13s 33ms/step - loss: 0.6801 - acc: 0.7100 - val_loss: 0.7156 - val_acc: 0.6924
Epoch 3/5
391/391 [==============================] - 12s 32ms/step - loss: 0.6282 - acc: 0.7325 - val_loss: 0.7038 - val_acc: 0.6916
Epoch 4/5
391/391 [==============================] - 13s 32ms/step - loss: 0.5932 - acc: 0.7451 - val_loss: 0.7160 - val_acc: 0.6981
Epoch 5/5
391/391 [==============================] - 13s 33ms/step - loss: 0.5652 - acc: 0.7557 - val_loss: 0.7177 - val_acc: 0.6938
